In [1]:
import pandas as pd
!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup
import requests
import numpy as np
from pandas.io.json import json_normalize  # JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#folium mapping
!conda install -c conda-forge folium=0.5.0 --yes
import folium


# convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [24]:
planp = pd.read_csv('planningpermission3.csv')
planp.head()

,_id,ApplicationNumber,Latitude,Longitude,OneOffHouseKPI,NumResidentialUnits,AreaOfSite,Decision
0,3,832737,51.875175,-8.496067,No,0,0.043,Refused
1,4,832738,51.898020,-8.414907,No,0,0.074,Granted (Conditional)
2,5,832739,51.883356,-8.482564,No,0,0.020,Granted (Conditional)
3,6,832740,51.887688,-8.473576,Yes,1,0.042,Refused
4,7,832741,51.917226,-8.469818,No,0,0.850,Granted (Conditional)


In [27]:
CLIENT_ID = 'TA1VTTRCMIEJT3SAICHXDWSNO1ZHSYLAUYWCUAKSKUCOBM20' # your Foursquare ID
CLIENT_SECRET = 'SB5FFECA3QYGAOCRZSUHO5BJXXRSVP3TTP5SUDVOAASDM0M2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TA1VTTRCMIEJT3SAICHXDWSNO1ZHSYLAUYWCUAKSKUCOBM20
CLIENT_SECRET:SB5FFECA3QYGAOCRZSUHO5BJXXRSVP3TTP5SUDVOAASDM0M2


In [28]:
application_latitude = planp.loc[0, 'Latitude'] # neighborhood latitude value
application_longitude = planp.loc[0, 'Longitude'] # neighborhood longitude value

application_id = planp.loc[0, 'ApplicationNumber'] # neighborhood name

#defining my Foursquare API
LIMIT = 10
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TA1VTTRCMIEJT3SAICHXDWSNO1ZHSYLAUYWCUAKSKUCOBM20&client_secret=SB5FFECA3QYGAOCRZSUHO5BJXXRSVP3TTP5SUDVOAASDM0M2&v=20180605&ll=51.87517519,-8.49606718&radius=500&limit=10'

In [29]:
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#now cleaning the data as was done for New York    

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Top / SPAR Express,Gas Station,51.874132,-8.495473
1,St Finbarrs H&F Club,Bar,51.877464,-8.496982
2,Centra,Convenience Store,51.872432,-8.498220
3,214 Bus Stop,Bus Stop,51.872400,-8.498315


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ApplicationNumber', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
Cork_venues = getNearbyVenues(names=planp['ApplicationNumber'],
                                   latitudes=planp['Latitude'],
                                   longitudes=planp['Longitude']
                                  )

832737
832738
832739
832740
832741
832742
832744
832745
832746
832747
832748
832749
832750
832751
832752
832753
832754
832755
832756
832757
832758
832760
832761
832762
832763
832765
832767
832768
832769
832770
832771
832772
832773
832774
832775
832776
832777
832778
832779
832780
832781
832782
832783
832784
832785
832787
832788
832789
832790
832791
832792
832793
832794
832795
832796
832797
832798
832801
832802
832803
832804
832805
832806
832807
832808
832809
832810
832811
832812
832813
832814
832815
832816
832817
832819
832820
832821
832824
832825
832826
832827
832828
832829
832830
832832
832833
832834
832836
832837
832838
832839
832840
832841
832842
832843
832844
832846
832847
832848
832849
832850
832851
832852
832853
832854
832855
832856
832857
832858
832859
832860
832861
832862
832865
832866
832867
832868
832869
832870
832871
832872
832873
832874
832875
832876
832877
832879
832880
832881
832882
832883
832884
832885
832886
832887
832888
832889
832890
832892
832893
832895
832896
832897

In [32]:
Cork_venues.head()

,ApplicationNumber,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,832737,51.875175,-8.496067,Top / SPAR Express,51.874132,-8.495473,Gas Station
1,832737,51.875175,-8.496067,St Finbarrs H&F Club,51.877464,-8.496982,Bar
2,832737,51.875175,-8.496067,Centra,51.872432,-8.498220,Convenience Store
3,832737,51.875175,-8.496067,214 Bus Stop,51.872400,-8.498315,Bus Stop
4,832738,51.898020,-8.414907,The Natural Foods Bakery,51.897363,-8.413323,Gourmet Shop


In [35]:
Cork_venues.groupby('ApplicationNumber').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
ApplicationNumber,,,,,,
832737,4,4,4,4,4,4
832738,5,5,5,5,5,5
832739,4,4,4,4,4,4
832740,4,4,4,4,4,4
832741,10,10,10,10,10,10
...,...,...,...,...,...,...
933688,5,5,5,5,5,5
933689,6,6,6,6,6,6
933690,4,4,4,4,4,4


In [38]:
# one hot encoding
cork_onehot = pd.get_dummies(Cork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cork_onehot['ApplicationNumber'] = Cork_venues['ApplicationNumber'] 

# move neighborhood column to the first column
fixed_columns = [cork_onehot.columns[-1]] + list(cork_onehot.columns[:-1])
cork_onehot = cork_onehot[fixed_columns]

cork_grouped = cork_onehot.groupby('ApplicationNumber').mean().reset_index()

num_top_venues = 5

for hood in cork_grouped['ApplicationNumber']:
    print(hood)
    temp = cork_grouped[cork_grouped['ApplicationNumber'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

832737
               venue  freq
0  Convenience Store  0.25
1        Gas Station  0.25
2                Bar  0.25
3           Bus Stop  0.25
4        Art Gallery  0.00


832738
             venue  freq
0            Trail   0.4
1     Gourmet Shop   0.2
2           Market   0.2
3  Harbor / Marina   0.2
4      Art Gallery   0.0


832739
                venue  freq
0               Plaza  0.25
1   Convenience Store  0.25
2  Chinese Restaurant  0.25
3         Supermarket  0.25
4                Park  0.00


832740
                  venue  freq
0        Soccer Stadium  0.25
1                  Park  0.25
2  Fast Food Restaurant  0.25
3     Fish & Chips Shop  0.25
4             Multiplex  0.00


832741
            venue  freq
0     Coffee Shop   0.2
1     Pizza Place   0.1
2  Shopping Plaza   0.1
3   Grocery Store   0.1
4   Shopping Mall   0.1


832742
                 venue  freq
0         Soccer Field  0.33
1          Pizza Place  0.33
2  Fried Chicken Joint  0.33
3          Art Gallery  0.00

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ApplicationNumber']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
application_venues_sorted = pd.DataFrame(columns=columns)
application_venues_sorted['ApplicationNumber'] = cork_grouped['ApplicationNumber']

for ind in np.arange(cork_grouped.shape[0]):
    application_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cork_grouped.iloc[ind, :], num_top_venues)

application_venues_sorted.head()

,ApplicationNumber,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,832737,Convenience Store,Bar,Gas Station,Bus Stop,Discount Store,Electronics Store,Farmers Market,Fast Food Restaurant,Department Store,Gastropub
1,832738,Trail,Market,Gourmet Shop,Harbor / Marina,Wine Shop,Field,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Court
2,832739,Plaza,Chinese Restaurant,Supermarket,Convenience Store,Department Store,Discount Store,Electronics Store,Farmers Market,Fast Food Restaurant,Gas Station
3,832740,Soccer Stadium,Park,Fish & Chips Shop,Fast Food Restaurant,Department Store,Discount Store,Electronics Store,Deli / Bodega,Farmers Market,Convenience Store
4,832741,Coffee Shop,Soba Restaurant,Restaurant,Shopping Mall,Fast Food Restaurant,Supermarket,Grocery Store,Pizza Place,Shopping Plaza,Department Store


In [42]:
#now time for clustering - we're using k means for 5 clusters here

# set number of clusters
kclusters = 5

cork_grouped_clustering = cork_grouped.drop('ApplicationNumber', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 3, 0, 1, 2, 2, 0, 4], dtype=int32)

In [49]:
# add clustering labels
application_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cork_merged = planp

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cork_merged = cork_merged.join(application_venues_sorted.set_index('ApplicationNumber'), on='ApplicationNumber')

cork_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [45]:
address = "Cork, Ireland"

geolocator = Nominatim(user_agent="cork_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Cork city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Cork city are 51.8979282, -8.4705806.


In [72]:
# finally we will create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cork_merged['Latitude'], cork_merged['Longitude'], cork_merged['ApplicationNumber'], cork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
manhattan_merged.loc[cork_merged['Cluster Labels'] == 0, cork_merged.columns[[1] + list(range(5, cork_merged.shape[1]))]]